# Анализ базы данных сервиса для чтения книг

**Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.<br> 
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. <br> 
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти книги помогут сформулировать ценностное предложение для нового продукта.**

**Описание данных:**<br>
*Таблица books*
Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

*Таблица authors*
Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

*Таблица publishers*
Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

*Таблица ratings*
Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

*Таблица reviews*
Содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


## Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

# устанавливаем параметры
db_config = {
    'user': 'praktikum_student',  # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp',       # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,                  # порт подключения
    'db': 'data-analyst-final-project-db'  # название базы данных
}

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode': 'require'})

# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query: str, engine: sa.engine.base.Engine = engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con=con)

## Ознакомление с содержанием таблиц

In [2]:
# Исследование таблиц
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

for table in tables:
    # Вывод первых строк таблицы
    query_first_rows = f'SELECT * FROM {table} LIMIT 5'
    table_data = get_sql_data(query_first_rows)
    print(f'\nПервые строки таблицы {table}:')
    display(table_data)

    # Подсчет количества строк в таблице
    query_row_count = f'SELECT COUNT(*) FROM {table}'
    row_count = get_sql_data(query_row_count).iloc[0, 0]
    print(f'\nКоличество строк в таблице {table}: {row_count}\n')


Первые строки таблицы books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



Количество строк в таблице books: 1000


Первые строки таблицы authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



Количество строк в таблице authors: 636


Первые строки таблицы publishers:


...


Количество строк в таблице publishers: 340


Первые строки таблицы ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



Количество строк в таблице ratings: 6456


Первые строки таблицы reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



Количество строк в таблице reviews: 2793



## Выполнение SQL-запросов

In [3]:
# формируем запрос и выводим данные
# Посчитаем, сколько книг вышло после 1 января 2000 года;
book_count_query = ''' SELECT COUNT(book_id)
                     FROM books
                     WHERE publication_date > '2000-01-01' '''

get_sql_data(book_count_query)

,count
0,819


**Итак, после 1 января 2000 года вышло 819 книг.**

In [4]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку
reviews_and_ratings_query = ''' 
SELECT
    b.book_id,
    b.title AS book_title,
    COUNT(DISTINCT r.review_id) AS review_count,
    AVG(rt.rating) AS avg_rating
FROM
    books b
LEFT JOIN
    reviews r ON b.book_id = r.book_id
LEFT JOIN
    ratings rt ON b.book_id = rt.book_id
GROUP BY
    b.book_id, b.title
ORDER BY
    book_id '''
get_sql_data(reviews_and_ratings_query)

,book_id,book_title,review_count,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


**В результате выполнения данного запроса мы получили таблицу с id и названием каждой книги из нашей базы данных с информацией о количестве обзоров на каждую книгу и её средним рейтингом.**

In [5]:
# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
thick_books_query = '''
SELECT
    p.publisher_id,
    p.publisher AS publisher_name,
    COUNT(b.book_id) AS book_count
FROM
    publishers p
LEFT JOIN
    books b ON p.publisher_id = b.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher_id, p.publisher
ORDER BY
    book_count DESC
LIMIT 1 '''
get_sql_data(thick_books_query)

,publisher_id,publisher_name,book_count
0,212,Penguin Books,42


**Издательство "Penguin Books" выпустило наибольшее количество книг толще 50 страниц - а именно 42 штуки.**

In [6]:
# Определим автора с самой высокой средней оценкой книг — будем учитывать только книги с 50 и более оценками
popular_author_query = '''
SELECT
    a.author_id,
    a.author,
    AVG(r.rating) AS avg_rating
FROM
    authors a
JOIN (
    SELECT
        b.author_id,
        b.book_id
    FROM
        books b
    JOIN
        ratings r ON b.book_id = r.book_id
    GROUP BY
        b.author_id, b.book_id
    HAVING
        COUNT(r.rating_id) >= 50
) rated_books ON a.author_id = rated_books.author_id
LEFT JOIN
    ratings r ON rated_books.book_id = r.book_id
GROUP BY
    a.author_id, a.author
ORDER BY
    avg_rating DESC
LIMIT 1 '''
get_sql_data(popular_author_query)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


**Мы установили, что самая высокая средняя оценка (учитывая книги только с 50 и более оценками) у авторов J.K. Rowling/Mary GrandPré, и она составляет почти 4.3.**

In [7]:
# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок
avg_reviews_num = '''
SELECT AVG(review_count) AS avg_reviews
FROM (
    SELECT r.username, COUNT(r.review_id) AS review_count
    FROM reviews r
    JOIN (
        SELECT rg.username
        FROM ratings rg
        GROUP BY rg.username
        HAVING COUNT(rg.rating_id) > 48
    ) rated_users ON r.username = rated_users.username
    GROUP BY r.username
) subquery '''
get_sql_data(avg_reviews_num)

,avg_reviews
0,24.0


**Среднее количество обзоров от пользователей, поставивших больше 48 оценок, составило 24.**

**В результате краткого ознакомительного исследования базы данных сервиса для чтения книг нами было установлено следующее:**<br>

**1. После 1 января 2000 года вышло 819 книг;**<br>
**2. Для ознакомления коллегами с информацией по книгам мы выгрузили таблицу с id и названием каждой книги из нашей базы данных с информацией о количестве обзоров на каждую книгу и её средним рейтингом;**<br>
**3. Издательство "Penguin Books" выпустило наибольшее количество книг толще 50 страниц - а именно 42 штуки;**<br>
**4. Самая высокая средняя оценка (учитывая книги только с 50 и более оценками) у авторов J.K. Rowling/Mary GrandPré, и она составляет почти 4.3;**<br>
**5. Среднее количество обзоров от пользователей, поставивших больше 48 оценок, составило 24.**